In [14]:
import torch
import polars as pl
from pathlib import Path
device = torch.device(
    "mps" if torch.backends.mps.is_available() 
    else "cuda" if torch.cuda.is_available() 
    else "cpu"
)
print(device)


mps


In [ ]:
pos_path = '../data/processed/methylated_cg_32context_small.parquet'
neg_path = '../data/processed/unmethylated_cg_32context_small.parquet'
pos_df = pl.read_parquet(pos_path)
neg_df = pl.read_parquet(neg_path)

In [16]:
pos_df.head()

read_name,cg_pos,window_seq,window_fi,window_fp,window_ri,window_rp
str,i64,str,list[i64],list[i64],list[i64],list[i64]
"""m64168_200823_191315/30/ccs""",15,"""AAGCAGTGGTATCAACGCAGAGTACTGGAA…","[11, 41, … 14]","[17, 18, … 17]","[23, 22, … 33]","[15, 23, … 27]"
"""m64168_200823_191315/30/ccs""",493,"""AGAGAAAGGCCAGGTCGCCTACAAAGGGAA…","[23, 27, … 27]","[32, 20, … 36]","[33, 43, … 30]","[25, 25, … 12]"
"""m64168_200823_191315/30/ccs""",1164,"""TCTCTTAACCTTTATCGATCTAGTCATAAT…","[28, 16, … 21]","[10, 27, … 19]","[7, 24, … 16]","[26, 18, … 30]"
"""m64168_200823_191315/30/ccs""",1786,"""AGGCACAAGATAGTGCGATTGTATCCCTGA…","[19, 49, … 21]","[13, 28, … 13]","[14, 36, … 52]","[14, 11, … 14]"
"""m64168_200823_191315/30/ccs""",1810,"""CCCTGAATAACCTTACGGAACTAAACTACC…","[17, 17, … 12]","[42, 35, … 20]","[12, 13, … 27]","[16, 14, … 23]"


In [12]:
neg_df.head()

read_name,cg_pos,window_seq,window_fi,window_fp,window_ri,window_rp
str,i64,str,list[i64],list[i64],list[i64],list[i64]
"""m64168_200820_000733/0/ccs""",15,"""AAGCAGTGGTATCAACGCAGAGTACTCATA…","[39, 45, … 25]","[19, 17, … 19]","[19, 57, … 18]","[13, 36, … 27]"
"""m64168_200820_000733/0/ccs""",116,"""ATGGCCAGGTGGTGGCGCCAAGGTCGTCCA…","[60, 31, … 69]","[12, 14, … 28]","[45, 45, … 41]","[24, 14, … 11]"
"""m64168_200820_000733/0/ccs""",125,"""TGGTGGCGCCAAGGTCGTCCAGCTATTTAT…","[26, 28, … 18]","[24, 15, … 32]","[17, 20, … 122]","[18, 20, … 17]"
"""m64168_200820_000733/0/ccs""",343,"""CTTTGTGAGATAGAGCGATAGTGATTTATA…","[21, 79, … 29]","[45, 11, … 19]","[52, 77, … 21]","[17, 27, … 27]"
"""m64168_200820_000733/0/ccs""",548,"""ACTCAGGATCAAACTCGTGCCAAACCTGTA…","[42, 32, … 44]","[15, 16, … 16]","[29, 21, … 88]","[14, 28, … 15]"


In [ ]:
class MethylDataset(Dataset):
    def __init__(
              self, 
              pos_path: Path, 
              neg_path: Path, 
              transform=None
              ):
           '''
           Arguments:
           pos_path: a path to a parquet dataframe with the positive (methylated) samples
           neg_path: a path to a parquet dataframe with the negative (unmethylated) samples
           transform: an optional transform callable on a single sample
           '''
        self.tranform = transform